# A Retrieval-Augmented QA System for Dungeons & Dragons

## 0. Installation

The following Python packages and libraries are required to execute the code.

In [1]:
import requests
import pprint
import json
from bs4 import BeautifulSoup
import re
import time

## 1. Data Crawling

### 1.1 Crawling and Structuring Raw API Data

This code saves the data tables from the *D&D 5e SRD API* [1] into the `api_data/api_data.json` file to preserve the original structure and metadata and obtain independence from the API and its availability. 

[1] 5e-bits. 2025. *D&D 5e API*. https://www.dnd5eapi.co/ (accessed: 19.09.2025).

Please note that the API does not include the changes from the latest edition published in 2024 as well as all the non-SRD data, and grave changes to the API might cause the code not to work. Therefore, as the `api_data.json` file already exists in the project directory and the underlying API may change in the future (which could cause the code to break), this code does NOT need to be run in order for `main.ipynb` to work.

The following specifies the file path of the JSON document in which the retrieved API data is stored.

In [2]:
file_path = 'api_data/api_data.json'

As some of the textual entries in the API contain unwanted characters such as `#`, `\n`, multiple whitespaces, or HTML tags, they first need to be cleaned.

The function `preprocess_text` takes an input text and converts it into a single string in case it is a list of strings. Then it removes any HTML or XML tags using `BeautifulSoup` as well as unwanted characters such as `#`, `_`, `*`, `(`, `)`, and line breaks using regular expressions. It reduces multiple consecutive whitespaces, repeated `-`, and `-` followed by a whitespace to a single whitespace as well as replacing multiple `|` with `,` and normalizing repeated commas. In the end, the final preprocessed text is returned.

Lowercasing, stopword removal, and punctuation stripping are deliberately avoided to keep the semantic structure of the descriptions which is important for later use with a language model as it needs to restructure the given text into a response.

In [3]:
def preprocess_text(text):
    if isinstance(text,list):
        text = ' '.join(text)  
    
    cleaned_text = BeautifulSoup(text, 'html.parser').text

    cleaned_text = re.sub(r'[#_*\(\n)]', ' ', cleaned_text)
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)
    cleaned_text = re.sub(r'(-{2,})|(-\s)', ' ', cleaned_text)
    cleaned_text = re.sub(r'\|+', ',', cleaned_text)
    final_text = re.sub(r'\s*,\s*(,\s*)+', ', ', cleaned_text)
    return final_text.strip()

As there is a specific rate limit of 10,000 API requests per second [2] that has to be respected, a maximum rate limit including a buffer is integrated into every subcategory API request call.

[2] 5e-bits. 2025. *FAQ*. https://5e-bits.github.io/docs/faq (accessed: 19.09.2025).

In [4]:
max_requests_per_second = 5000
delay = 1 / max_requests_per_second

The code for crawling the data from the API is inspired by the tutorial on the API's website [3].

[3] 5e-bits. 2025. *Command Line Spellbook with Python*. https://5e-bits.github.io/docs/tutorials/advanced/terminal-spellbook-with-python (accessed: 19.09.2025).

The following code was employed to get an understanding of the access process. Here, no data is returned, as the class `barbarian` does not contain any `spellcasting` information. This illustrates the necessity of refining the API requests for each specific endpoint.

In [ ]:
url = 'https://www.dnd5eapi.co/api/2014/classes/barbarian/spellcasting'

payload = {}
headers = {
  'Accept': 'application/json'
}

response = requests.request('GET', url, headers=headers, data=payload)
answer_whole = response.text
print(answer_whole)

{"error":"Not found"}


As documented in the *Command Line Spellbook* [3], the following link provides general access to the API: https://www.dnd5eapi.co/api/2014/. When opened in a web browser, the endpoint returns an overview of the available tables together with their respective links. In the subsequent steps, each table is accessed and stored within dictionaries, which are then used to generate a comprehensive JSON file containing the retrieved data.

The API endpoints for `conditions`, `damage-types`, `magic-schools`, and `weapon-properties` all share the same structure, which allows to use a single function, `create_dict`, to process them. This function takes the endpoint type as input, fetches the list of entries, and then retrieves the detailed data for each entry. From each result, the `name` and processed `desc` (description) fields are extracted and stored in a dictionary, keyed by the entry's index. The function then returns the completed dictionary. Finally, separate dictionaries are generated for each of the four categories, with the `conditions` dictionary shown as example output.

In [ ]:
list_of_indices = ['conditions','damage-types','magic-schools','weapon-properties']

dict_of_conditions = {}
dict_of_damage_types = {}
dict_of_magic_schools = {}
dict_of_weapon_properties = {}

def create_dict(type):

    dict_of_response_data = {}

    time.sleep(delay)
    
    url = 'https://www.dnd5eapi.co/api/2014/'+type
    response = requests.request('GET', url, headers=headers, data=payload)
    resp = response.json()

    for entry in resp['results']:
        response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
        resp2 = response2.json()

        response_data = {
            'name': entry['name'].lower(),
            'desc': ''.join(preprocess_text(resp2['desc']))
        }

        dict_of_response_data[entry['index']] = response_data
    return dict_of_response_data

dict_of_conditions = create_dict(list_of_indices[0])
dict_of_damage_types = create_dict(list_of_indices[1])
dict_of_magic_schools = create_dict(list_of_indices[2])
dict_of_weapon_properties = create_dict(list_of_indices[3])

pprint.pprint(dict_of_conditions)

{'blinded': {'desc': "A blinded creature can't see and automatically fails any "
                     'ability check that requires sight.  Attack rolls against '
                     "the creature have advantage, and the creature's attack "
                     'rolls have disadvantage.',
             'name': 'blinded'},
 'charmed': {'desc': "A charmed creature can't attack the charmer or target "
                     'the charmer with harmful abilities or magical effects.  '
                     'The charmer has advantage on any ability check to '
                     'interact socially with the creature.',
             'name': 'charmed'},
 'deafened': {'desc': "A deafened creature can't hear and automatically fails "
                      'any ability check that requires hearing.',
              'name': 'deafened'},
 'exhaustion': {'desc': 'Some special abilities and environmental hazards, '
                        'such as starvation and the long-term effects of '
                  

Next, the remaining API endpoints are processed in a manner analogous to the previously described procedure. First, a request is sent to fetch the list of entries, and an empty dictionary is initialized to store the results. For each entry, the `name` attribute is extracted, the complete details are retrieved via a secondary request, and the description is standardized using the `preprocess_text` function. Both the `name` and the cleaned `desc` are then stored into a dictionary. Only information relevant to the intended QA system is retained; for instance, attributes like `url` are disregarded, as they are not essential. Optional fields including `races`, `subraces`, and `proficiencies` are incorporated only if present in the API response, since not all endpoints provdide values for these attributes. If a response is associated with specific races or subraces, an additional flag `exclusive_to_stated_races` is set to `True`. Finally, each entry is inserted into a dictionary using its `index` as the key.

First, the API endpoint `traits` is processed.

In [7]:
url = 'https://www.dnd5eapi.co/api/2014/traits'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_traits = {}
trait_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(delay)
    
     response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
     resp2 = response2.json()

     trait_data = {
          'name': name,
          'desc': ''.join(preprocess_text(resp2['desc']))
     }
     
     if resp2.get('races'):
          trait_data['races'] = [item['name'] for item in resp2['races']]

     if resp2.get('subraces'):
          trait_data['subraces'] = [item['name'] for item in resp2['subraces']]

     if resp2.get('proficiencies'):
          trait_data['proficiencies'] = [item['name'] for item in resp2['proficiencies']]
     
     if resp2.get('races') or resp2.get('subraces'):
          trait_data['exclusive_to_stated_races'] = True
     
     dict_of_traits[entry['index']] = trait_data

pprint.pprint(dict_of_traits)

{'artificers-lore': {'desc': 'Whenever you make an Intelligence History check '
                             'related to magic items, alchemical objects, or '
                             'technological devices, you can add twice your '
                             'proficiency bonus, instead of any proficiency '
                             'bonus you normally apply.',
                     'exclusive_to_stated_races': True,
                     'name': "Artificer's Lore",
                     'subraces': ['Rock Gnome']},
 'brave': {'desc': 'You have advantage on saving throw against being '
                   'frightened.',
           'exclusive_to_stated_races': True,
           'name': 'Brave',
           'races': ['Halfling']},
 'breath-weapon': {'desc': 'You can use your action to exhale destructive '
                           'energy. Your draconic ancestry determines the '
                           'size, shape, and damage type of the exhalation. '
                           '

Here, the API endpoint `rule-sections` is processed.

In [8]:
url = 'https://www.dnd5eapi.co/api/2014/rule-sections'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_rule_sections = {}
section_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)

    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    section_data = {
        'name': name,
        'desc': preprocess_text(resp2['desc']),
    }
    
    dict_of_rule_sections[entry['index']] = section_data

pprint.pprint(dict_of_rule_sections)

{'ability-checks': {'desc': 'Ability Checks An ability check tests a '
                            "character's or monster's innate talent and "
                            'training in an effort to overcome a challenge. '
                            'The GM calls for an ability check when a '
                            'character or monster attempts an action other '
                            'than an attack that has a chance of failure. When '
                            'the outcome is uncertain, the dice determine the '
                            'results. For every ability check, the GM decides '
                            'which of the six abilities is relevant to the '
                            'task at hand and the difficulty of the task, '
                            'represented by a Difficulty Class. The more '
                            'difficult a task, the higher its DC. The Typical '
                            'Difficulty Classes table shows the most common '
 

Next, the API endpoint `skills` is processed. 

In [9]:
url = 'https://www.dnd5eapi.co/api/2014/skills'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_skills = {}
skill_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)

    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    skill_data = {
        'name': name,
        'desc': preprocess_text(resp2['desc']),
        'ability_score': resp2['ability_score']['name']
    }
     
    dict_of_skills[entry['index']] = skill_data

pprint.pprint(dict_of_skills)

{'acrobatics': {'ability_score': 'DEX',
                'desc': 'Your Dexterity Acrobatics check covers your attempt '
                        'to stay on your feet in a tricky situation, such as '
                        "when you're trying to run across a sheet of ice, "
                        'balance on a tightrope, or stay upright on a rocking '
                        "ship's deck. The GM might also call for a Dexterity "
                        'Acrobatics check to see if you can perform acrobatic '
                        'stunts, including dives, rolls, somersaults, and '
                        'flips.',
                'name': 'Acrobatics'},
 'animal-handling': {'ability_score': 'WIS',
                     'desc': 'When there is any question whether you can calm '
                             'down a domesticated animal, keep a mount from '
                             "getting spooked, or intuit an animal's "
                             'intentions, the GM might call for 

Furthermore, the API endpoint `feats` is processed. 

In [10]:
url = 'https://www.dnd5eapi.co/api/2014/feats'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_feats = {}
feat_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(delay)

     response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
     resp2 = response2.json()

     feat_data = {
          'name': name,
          'desc': preprocess_text(resp2['desc'])
     }
     if resp2.get('prerequisites'):
          feat_data['prerequisites']= [{'ability_score': item['ability_score']['name'], 'minimum_score': item['minimum_score']} for item in resp2['prerequisites']]
     
     dict_of_feats[entry['index']] = feat_data

pprint.pprint(dict_of_feats)

{'grappler': {'desc': 'You’ve developed the Skills necessary to hold your own '
                      'in close quarters Grappling. You gain the following '
                      'benefits:  You have advantage on Attack Rolls against a '
                      'creature you are Grappling.  You can use your action to '
                      'try to pin a creature Grappled by you. To do so, make '
                      'another grapple check. If you succeed, you and the '
                      'creature are both Restrained until the grapple ends.',
              'name': 'Grappler',
              'prerequisites': [{'ability_score': 'STR', 'minimum_score': 13}]}}


Additionally, the API endpoint `ability-scores` is processed.

In [11]:
url = 'https://www.dnd5eapi.co/api/2014/ability-scores'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_ability_scores = {}
ability_score_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(delay)
     
     response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
     resp2 = response2.json()

     ability_score_data = {
         'abbreviation': name,
         'name': resp2['full_name'],
         'desc': preprocess_text(resp2['desc'])
     }

     if resp2.get('skills'):
          ability_score_data['skills']= [item['name'] for item in resp2['skills']]
     
     dict_of_ability_scores[entry['index']] = ability_score_data

pprint.pprint(dict_of_ability_scores)

{'cha': {'abbreviation': 'CHA',
         'desc': 'Charisma measures your ability to interact effectively with '
                 'others. It includes such factors as confidence and '
                 'eloquence, and it can represent a charming or commanding '
                 'personality. A Charisma check might arise when you try to '
                 'influence or entertain others, when you try to make an '
                 'impression or tell a convincing lie, or when you are '
                 'navigating a tricky social situation. The Deception, '
                 'Intimidation, Performance, and Persuasion skills reflect '
                 'aptitude in certain kinds of Charisma checks.',
         'name': 'Charisma',
         'skills': ['Deception', 'Intimidation', 'Performance', 'Persuasion']},
 'con': {'abbreviation': 'CON',
         'desc': 'Constitution measures health, stamina, and vital force. '
                 'Constitution checks are uncommon, and no skills apply to '
    

Here, the API endpoint `languages` is processed.

In [12]:
url = 'https://www.dnd5eapi.co/api/2014/languages'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_languages = {}
language_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(delay)
     
     response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
     resp2 = response2.json()

     language_data = {
         'name': name,
         'type': resp2['type'],
         'typical_speakers': resp2['typical_speakers']
     }

     if resp2.get('script'):
          language_data['script']= resp2['script']
     
     dict_of_languages[entry['index']] = language_data

pprint.pprint(dict_of_languages)

{'abyssal': {'name': 'Abyssal',
             'script': 'Infernal',
             'type': 'Exotic',
             'typical_speakers': ['Demons']},
 'celestial': {'name': 'Celestial',
               'script': 'Celestial',
               'type': 'Exotic',
               'typical_speakers': ['Celestials']},
 'common': {'name': 'Common',
            'script': 'Common',
            'type': 'Standard',
            'typical_speakers': ['Humans']},
 'deep-speech': {'name': 'Deep Speech',
                 'type': 'Exotic',
                 'typical_speakers': ['Aboleths', 'Cloakers']},
 'draconic': {'name': 'Draconic',
              'script': 'Draconic',
              'type': 'Exotic',
              'typical_speakers': ['Dragons', 'Dragonborn']},
 'dwarvish': {'name': 'Dwarvish',
              'script': 'Dwarvish',
              'type': 'Standard',
              'typical_speakers': ['Dwarves']},
 'elvish': {'name': 'Elvish',
            'script': 'Elvish',
            'type': 'Standard',
         

Subsequently, the API endpoint `classes` is processed. Each class entry provides an associated link to the `/levels` endpoint, for example: https://www.dnd5eapi.co/api/2014/classes/barbarian/levels. 
For every entry, the respective level changes are recorded in the `level_changes` list and then stored under the key `class_levels` within the `class_data` structure.

In [13]:
url = 'https://www.dnd5eapi.co/api/2014/classes'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_classes = {}
class_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)

    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    class_data = {
        'name': name,
        'hit_die': resp2['hit_die']
    }
    
    if resp2.get('proficiency_choices'):
        class_data['proficiency_choices'] = preprocess_text([item['desc'] for item in resp2['proficiency_choices']])

    if resp2.get('proficiencies'):
        class_data['proficiencies'] = [item['name'] for item in resp2['proficiencies']]

    if resp2.get('saving_throws'):
        class_data['saving_throws'] = [item['name'] for item in resp2['saving_throws']]
     
    if resp2.get('starting_equipment'):
        class_data['starting_equipment'] = [{'name': item['equipment']['name'], 'quantity': item['quantity']} for item in resp2['starting_equipment']]
     
    if resp2.get('starting_equipment_options'):
        class_data['starting_equipment_options'] = preprocess_text([item['desc'] for item in resp2['starting_equipment_options']])
     

    time.sleep(delay)
     
    response3 = requests.request('GET', url+f'/{entry['index']}/levels', headers=headers, data=payload)
    resp3 = response3.json()

    level_changes = []
    
    for lvl_entries in resp3:
        level_dict = {
            'level': lvl_entries['level'],
            'ability_score_bonuses': lvl_entries['ability_score_bonuses'],
            'proficienciy_bonus': lvl_entries['prof_bonus'],
            'features': [item['name'] for item in lvl_entries['features']],
            'class_specific': lvl_entries['class_specific']
        }
        if lvl_entries.get('spellcasting'):
            level_dict['spellcasting'] = lvl_entries['spellcasting']
        level_changes.append(level_dict)

    if resp2.get('class_levels'):
        class_data['class_levels'] = level_changes
     
    if resp2.get('multi_classing'):
        multi_class_dict = {}
        if resp2['multi_classing'].get('prerequisites'):
            prerequisites = [{'ability': item['ability_score']['name'], 'minimum_score': item['minimum_score']} for item in resp2['multi_classing']['prerequisites']]
            multi_class_dict['prerequisites'] = prerequisites
        if resp2['multi_classing'].get('proficiencies'):
            multi_class_dict['proficienies'] = [item['name'] for item in resp2['multi_classing']['proficiencies']]
        class_data['multi_classing'] = [multi_class_dict]
     
    if resp2.get('subclasses'):
        class_data['subclasses'] = [item['name'] for item in resp2['subclasses']]

    time.sleep(delay)

    response3 = requests.request('GET', url+f'/{entry['index']}/spellcasting', headers=headers, data=payload)
    if response3.status_code == 200:
        resp3 = response3.json()
        spellcasting_dict = {}
        spellcasting_dict['level'] = resp3['level']
        spellcasting_dict['spellcasting_ability'] = resp3['spellcasting_ability']['name']
        spellcasting_dict['info'] = [{'name': item['name'], 'desc': preprocess_text(item['desc'])} for item in resp3['info']]
        class_data['spellcasting_info'] = spellcasting_dict
    else:
        print(f'No spellcasting info available for {entry['name']}.')

    time.sleep(delay)

    response4 = requests.request('GET', url+f'/{entry['index']}/spells', headers=headers, data=payload)
    resp4 = response4.json()
    if resp4['count']!= 0:
        spell_dict = {}
        for info in resp4['results']:
            spell_dict[info['name']] = {'needed_level': info['level']}
        class_data['available_spells_for_class'] = spell_dict

    dict_of_classes[entry['index']] = class_data

pprint.pprint(dict_of_classes)

No spellcasting info available for Barbarian.
No spellcasting info available for Fighter.
No spellcasting info available for Monk.
No spellcasting info available for Rogue.
{'barbarian': {'class_levels': [{'ability_score_bonuses': 0,
                                 'class_specific': {'brutal_critical_dice': 0,
                                                    'rage_count': 2,
                                                    'rage_damage_bonus': 2},
                                 'features': ['Rage', 'Unarmored Defense'],
                                 'level': 1,
                                 'proficienciy_bonus': 2},
                                {'ability_score_bonuses': 0,
                                 'class_specific': {'brutal_critical_dice': 0,
                                                    'rage_count': 2,
                                                    'rage_damage_bonus': 2},
                                 'features': ['Reckless Attack',
          

Furthermore, the API endpoint `subclasses` is processed.

In [14]:
url = 'https://www.dnd5eapi.co/api/2014/subclasses'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_subclasses = {}
subclass_data = {}

for entry in resp['results']:
     name = entry['name']

     time.sleep(delay)

     response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
     resp2 = response2.json()

     subclass_data = {
         'name': name,
         'class': resp2['class']['name'],
         'subclass_flavor': resp2['subclass_flavor'],
         'desc': preprocess_text(resp2['desc'])
     }
     
     time.sleep(delay)
     
     response3 = requests.request('GET', url+f'/{entry['index']}/levels', headers=headers, data=payload)
     resp3 = response3.json()

     sublevel_changes = []

     for lvl_entries in resp3:
          sublevel_dict = {
               'level': lvl_entries['level'],
               'features': [item['name'] for item in lvl_entries['features']],
               'class': lvl_entries['class']
          }
          sublevel_changes.append(level_dict)

     if resp2.get('subclass_levels'):
          subclass_data['subclass_levels'] = level_changes

     dict_of_subclasses[entry['index']] = subclass_data

pprint.pprint(dict_of_subclasses)

{'berserker': {'class': 'Barbarian',
               'desc': 'For some barbarians, rage is a means to an end that '
                       'end being violence. The Path of the Berserker is a '
                       'path of untrammeled fury, slick with blood. As you '
                       "enter the berserker's rage, you thrill in the chaos of "
                       'battle, heedless of your own health or well-being.',
               'name': 'Berserker',
               'subclass_flavor': 'Primal Path',
               'subclass_levels': [{'ability_score_bonuses': 0,
                                    'class_specific': {'arcane_recovery_levels': 1},
                                    'features': ['Spellcasting: Wizard',
                                                 'Arcane Recovery'],
                                    'level': 1,
                                    'proficienciy_bonus': 2,
                                    'spellcasting': {'cantrips_known': 3,
              

In addition, the API endpoint `rules` is processed. In certain cases, the variables stored within the tables contain lists of dictionaries. Each dictionary is therefore accessed individually, and the `name` attribute of the corresponding subsection is stored in a list.

In [15]:
url = 'https://www.dnd5eapi.co/api/2014/rules'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_rules = {}
rule_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    rule_data = {
        'name' : name,
        'desc': ''.join(preprocess_text(resp2['desc']))
    }

    if resp2.get('subsections'):
        rule_data['subsection_in_rule_sections'] = [item['name'] for item in resp2['subsections']]

    dict_of_rules[entry['index']] = rule_data

pprint.pprint(dict_of_rules)

{'adventuring': {'desc': 'Adventuring',
                 'name': 'Adventuring',
                 'subsection_in_rule_sections': ['Time',
                                                 'Movement',
                                                 'The Environment',
                                                 'Traps',
                                                 'Diseases',
                                                 'Madness',
                                                 'Resting',
                                                 'Between Adventures']},
 'appendix': {'desc': 'Appendix',
              'name': 'Appendix',
              'subsection_in_rule_sections': ['Fantasy-Historical Pantheons',
                                              'The Planes of Existence']},
 'combat': {'desc': 'Combat',
            'name': 'Combat',
            'subsection_in_rule_sections': ['The Order of Combat',
                                            'Movement and Position',
     

Here, the API endpoint `spells` is processed.

In [16]:
url = 'https://www.dnd5eapi.co/api/2014/spells'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_spells = {}
spell_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    spell_data = {
        'name' : name,
        'desc': ''.join(preprocess_text(resp2['desc'])),
        'range': resp2['range'],
        'components':resp2['components'],
        'ritual':resp2['ritual'],
        'duration': resp2['duration'],
        'concentration': resp2['concentration'],
        'casting_time': resp2['casting_time'],
        'level': resp2['level'],
        'school_of_magic': [resp2['school']['name']],
        'classes': [item['name'] for item in resp2['classes']],
    }

    if resp2.get('material'):
        spell_data['material'] = resp2['material']

    if resp2.get('subclasses'):
        spell_data['subclasses'] = [item['name'] for item in resp2['subclasses']]

    if resp2.get('higher_level'):
        spell_data['higher_level'] = resp2['higher_level']

    if resp2.get('damage'):
        slot_level_damage = {}
        if 'damage_type' in resp2['damage']:
            spell_data['damage_type'] = [resp2['damage']['damage_type']['name']]
        if 'damage_at_slot_level' in resp2['damage']:
            for slot_level, damage in resp2['damage']['damage_at_slot_level'].items():
                slot_level_damage[slot_level] = damage
            spell_data['damage_at_slot_level'] = slot_level_damage

    if resp2.get('classes') or resp2.get('subclasses'):
        spell_data['exclusive_to_stated_classes'] = True

    if resp2.get('attack_type'):
       spell_data['attack_type'] = resp2['attack_type']
    
    dict_of_spells[entry['index']] = spell_data

pprint.pprint(dict_of_spells)

{'acid-arrow': {'attack_type': 'ranged',
                'casting_time': '1 action',
                'classes': ['Wizard'],
                'components': ['V', 'S', 'M'],
                'concentration': False,
                'damage_at_slot_level': {'2': '4d4',
                                         '3': '5d4',
                                         '4': '6d4',
                                         '5': '7d4',
                                         '6': '8d4',
                                         '7': '9d4',
                                         '8': '10d4',
                                         '9': '11d4'},
                'damage_type': ['Acid'],
                'desc': 'A shimmering green arrow streaks toward a target '
                        'within range and bursts in a spray of acid. Make a '
                        'ranged spell attack against the target. On a hit, the '
                        'target takes 4d4 acid damage immediately and 2d4 acid '
     

Next, the API endpoint `races` is processed.

In [17]:
url = 'https://www.dnd5eapi.co/api/2014/races'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_races = {}
race_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)

    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus']
    
    race_data = {
        'name' : name,
        'speed': resp2['speed'],
        'ability_bonuses': bonus_dict,
        'alignment': resp2['alignment'],
        'age': resp2['age'],
        'size': resp2['size'],
        'size_description': resp2['size_description'],
        'languages': [item['name'] for item in resp2['languages']],
        'language_description': resp2['language_desc'],
        'traits': [item['name'] for item in resp2['traits']],
    }

    if resp2.get('subraces') :
        race_data['subraces'] = [item['name'] for item in resp2['subraces']]

    if resp2.get('starting_proficiencies'):
        race_data['starting_proficiencies'] =  [item['name'] for item in resp2['starting_proficiencies']]
    
    dict_of_races[entry['index']] = race_data

pprint.pprint(dict_of_races)

{'dragonborn': {'ability_bonuses': {'CHA': 1, 'STR': 2},
                'age': 'Young dragonborn grow quickly. They walk hours after '
                       'hatching, attain the size and development of a '
                       '10-year-old human child by the age of 3, and reach '
                       'adulthood by 15. They live to be around 80.',
                'alignment': 'Dragonborn tend to extremes, making a conscious '
                             'choice for one side or the other in the cosmic '
                             'war between good and evil. Most dragonborn are '
                             'good, but those who side with evil can be '
                             'terrible villains.',
                'language_description': 'You can speak, read, and write Common '
                                        'and Draconic. Draconic is thought to '
                                        'be one of the oldest languages and is '
                                       

Furthermore, the API endpoint `subraces` is processed.

In [18]:
url = 'https://www.dnd5eapi.co/api/2014/subraces'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_subraces = {}
subraces_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    subraces_data = {
        'name' : name,
        'desc': ''.join(preprocess_text(resp2['desc'])),
        'race': resp2['race']['name'],
        'ability_bonuses': bonus_dict,
        'racial_traits': [item['name'] for item in resp2['racial_traits']],
    }

    if resp2.get('starting_proficiencies'):
        subraces_data['starting_proficiencies'] = [item['name'] for item in resp2['starting_proficiencies']]

    if resp2.get('languages'):
        subraces_data['languages'] = resp2['languages']

    if resp2.get('language_options'):
        languages = resp2['language_options']['from']['options']
        language_names = [lang['item']['name'] for lang in languages]
        subraces_data['language_options'] = language_names

    bonus_dict = {}
    for item in resp2['ability_bonuses']:
        bonus_dict[item['ability_score']['name']] = item['bonus'] 

    dict_of_subraces[entry['index']] = subraces_data
    
pprint.pprint(dict_of_subraces)

{'high-elf': {'ability_bonuses': {'CHA': 2, 'INT': 1},
              'desc': 'As a high elf, you have a keen mind and a mastery of at '
                      'least the basics of magic. In many fantasy gaming '
                      'worlds, there are two kinds of high elves. One type is '
                      'haughty and reclusive, believing themselves to be '
                      'superior to non-elves and even other elves. The other '
                      'type is more common and more friendly, and often '
                      'encountered among humans and other races.',
              'language_options': ['Dwarvish',
                                   'Giant',
                                   'Gnomish',
                                   'Goblin',
                                   'Halfling',
                                   'Orc',
                                   'Abyssal',
                                   'Celestial',
                                   'Draconic',
  

Additionally, the API endpoint `proficiencies` is processed.

In [19]:
url = 'https://www.dnd5eapi.co/api/2014/proficiencies'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_proficiencies = {}
proficiency_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    proficiency_data = {
        'name' : name,
        'type' : resp2['type'],
    }

    if resp2.get('classes'):
        proficiency_data['classes'] = [item['name'] for item in resp2['classes']]

    if resp2.get('races'):
        proficiency_data['races'] = [item['name'] for item in resp2['races']]
    
    if resp2.get('classes') or resp2.get('races'):
        proficiency_data['exclusive_to_stated_classes_and_races'] = True

    dict_of_proficiencies[entry['index']] = proficiency_data

pprint.pprint(dict_of_proficiencies)

{'alchemists-supplies': {'name': "Alchemist's Supplies",
                         'type': "Artisan's Tools"},
 'all-armor': {'classes': ['Fighter', 'Paladin'],
               'exclusive_to_stated_classes_and_races': True,
               'name': 'All armor',
               'type': 'Armor'},
 'bagpipes': {'name': 'Bagpipes', 'type': 'Musical Instruments'},
 'battleaxes': {'exclusive_to_stated_classes_and_races': True,
                'name': 'Battleaxes',
                'races': ['Dwarf'],
                'type': 'Weapons'},
 'blowguns': {'name': 'Blowguns', 'type': 'Weapons'},
 'breastplate': {'name': 'Breastplate', 'type': 'Armor'},
 'brewers-supplies': {'name': "Brewer's Supplies", 'type': "Artisan's Tools"},
 'calligraphers-supplies': {'name': "Calligrapher's Supplies",
                            'type': "Artisan's Tools"},
 'carpenters-tools': {'name': "Carpenter's Tools", 'type': "Artisan's Tools"},
 'cartographers-tools': {'name': "Cartographer's Tools",
                        

Here, the API endpoint `equipment` is processed.

In [20]:
url = 'https://www.dnd5eapi.co/api/2014/equipment'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()
dict_of_equipment = {}
equipment_data = {}

for entry in resp['results']:
    name_of_equip = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()
    
    equipment_data =  {
        'name' : name_of_equip,
        'equipment_category': resp2['equipment_category']['name'],
    }

    if resp2.get('desc'):
        equipment_data['desc'] = ''.join(preprocess_text(resp2['desc']))

    if resp2.get('weight'):
        equipment_data['weight'] = resp2['weight']

    if resp2.get('special'):
        equipment_data['special'] = resp2['special']

    if resp2.get('properties'):
        equipment_data['properties'] = [item['name'] for item in resp2['properties']]

    if resp2.get('contents'):
        equipment_data['contents'] = [{'name': item['item']['name']} for item in resp2['contents']]

    if resp2.get('gear-category'):
        equipment_data['gear-category'] = resp2['gear-category']['name']

    if resp2.get('armor_category'):
        equipment_data['armor_category'] = resp2['armor_category']
    
    if resp2.get('armor_class'):
        equipment_data['armor_class'] = {'base_value': resp2['armor_class']['base'], 'dexterity_bonus': resp2['armor_class']['dex_bonus'], 'max_bonus': resp2['armor_class'].get('armor_class', 'none')}
    
    if resp2.get('str_minimum'):
        equipment_data['str_minimum'] = resp2['str_minimum']
    
    if resp2.get('stealth_disadvantage'):
        equipment_data['stealth_disadvantage'] = resp2['stealth_disadvantage']

    if resp2.get('weapon_category'):
        equipment_data['weapon_category'] = resp2['weapon_category']

    if resp2.get('weapon_range'):
        equipment_data['weapon_range'] = resp2['weapon_range']
    
    if resp2.get('category_range'):
        equipment_data['category_range'] = resp2['category_range']
    
    if resp2.get('cost'):
        equipment_data['cost'] = {'amount': resp2['cost']['quantity'], 'unit': resp2['cost']['unit']}

    if resp2.get('damage'):
        equipment_data['damage'] = {'damage_dice': resp2['damage']['damage_dice'], 'damage_type':  resp2['damage']['damage_type']['name']}
    
    if resp2.get('range'): 
        equipment_data['range'] = resp2['range']

    if resp2.get('properties'):
        equipment_data['properties'] = [item['name'] for item in resp2['properties']]

    if resp2.get('tool_category'):
        equipment_data['tool_category'] = resp2['tool_category']

    if resp2.get('speed'):
        equipment_data['speed'] = resp2['speed']
    
    if resp2.get('capacity'):
        equipment_data['capacity'] = resp2['capacity']

    dict_of_equipment[entry['index']] = equipment_data

pprint.pprint(dict_of_equipment)

{'abacus': {'cost': {'amount': 2, 'unit': 'gp'},
            'equipment_category': 'Adventuring Gear',
            'name': 'Abacus',
            'weight': 2},
 'acid-vial': {'cost': {'amount': 25, 'unit': 'gp'},
               'desc': 'As an action, you can splash the contents of this vial '
                       'onto a creature within 5 feet of you or throw the vial '
                       'up to 20 feet, shattering it on impact. In either '
                       'case, make a ranged attack against a creature or '
                       'object, treating the acid as an improvised weapon. On '
                       'a hit, the target takes 2d6 acid damage.',
               'equipment_category': 'Adventuring Gear',
               'name': 'Acid (vial)',
               'weight': 1},
 'alchemists-fire-flask': {'cost': {'amount': 50, 'unit': 'gp'},
                           'desc': 'This sticky, adhesive fluid ignites when '
                                   'exposed to air. As an ac

Next, the API endpoint `features` is processed.

In [21]:
url =  'https://www.dnd5eapi.co/api/2014/features'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_features = {}
feature_data = {}

for entry in resp['results']:
    
    time.sleep(delay)

    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    feature_data = {
        'name' : entry['name'],
        'desc' : ''.join(preprocess_text(resp2['desc'])),
        'exclusive_to_class': resp2['class']['name'],
        'level': resp2['level']
    }

    if resp2.get('prerequisites'):
        feature_data['prerequisites'] =  resp2['prerequisites']
    
    dict_of_features[entry['index']] = feature_data

pprint.pprint(dict_of_features)

{'action-surge-1-use': {'desc': 'Starting at 2nd level, you can push yourself '
                                'beyond your normal limits for a moment. On '
                                'your turn, you can take one additional action '
                                'on top of your regular action and a possible '
                                'bonus action. Once you use this feature, you '
                                'must finish a short or long rest before you '
                                'can use it again. Starting at 17th level, you '
                                'can use it twice before a rest, but only once '
                                'on the same turn.',
                        'exclusive_to_class': 'Fighter',
                        'level': 2,
                        'name': 'Action Surge (1 use)'},
 'action-surge-2-uses': {'desc': 'Starting at 2nd level, you can push yourself '
                                 'beyond your normal limits for a moment. On

Additionally, the API endpoint `magic-items` is processed.

In [22]:
url = 'https://www.dnd5eapi.co/api/2014/magic-items'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_magic_items = {}
item_data = {}

for entry in resp['results']:
    name_of_item = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    item_data = {
        'name' : entry['name'],
        'desc' : ''.join(preprocess_text(resp2['desc'])),
        'equipment-category': resp2['equipment_category']['name'],
        'rarity': resp2['rarity']['name'],
    }

    if resp2.get('variants'):
        item_data['variants'] = [item['name'] for item in resp2['variants']]

    dict_of_magic_items[entry['index']] = item_data

pprint.pprint(dict_of_magic_items)

{'adamantine-armor': {'desc': 'Armor medium or heavy, but not hide , uncommon '
                              'This suit of armor is reinforced with '
                              'adamantine, one of the hardest substances in '
                              "existence. While you're wearing it, any "
                              'critical hit against you becomes a normal hit.',
                      'equipment-category': 'Armor',
                      'name': 'Adamantine Armor',
                      'rarity': 'Uncommon'},
 'ammunition': {'desc': 'Weapon any ammunition , uncommon +1 , rare +2 , or '
                        'very rare +3 You have a bonus to attack and damage '
                        'rolls made with this piece of magic ammunition. The '
                        'bonus is determined by the rarity of the ammunition. '
                        'Once it hits a target, the ammunition is no longer '
                        'magical.',
                'equipment-category': 'Am

Furthermore, the API endpoint `monsters` is processed.

In [23]:
url = 'https://www.dnd5eapi.co/api/2014/monsters'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_monsters = {}
monster_data = {}

for entry in resp['results']:
    name_of_monster = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    senses = {}
    for key, value in resp2['senses'].items():
        senses[key] = value
    
    movement = {}
    for key, value in resp2['speed'].items():
        movement[key] = value

    monster_data = {
        'name' : name_of_monster,
        'size': resp2['size'],
        'type':resp2['type'],
        'alignment': resp2['alignment'],
        'hit_points':resp2['hit_points'],
        'hit_dice':resp2['hit_dice'],
        'hit_points_roll': resp2['hit_points_roll'],
        'speed': movement,
        'strength': resp2['strength'],
        'dexterity':resp2['dexterity'],
        'constitution': resp2['constitution'],
        'intelligence': resp2['intelligence'],
        'wisdom': resp2['wisdom'],
        'charisma': resp2['charisma'],
        'senses': senses,
        'languages': resp2['languages'],
        'challenge_rating': resp2['challenge_rating'],
        'proficiency_bonus': resp2['proficiency_bonus'],
        'gained_experience': resp2['xp']
    }

    if resp2.get('armor_class'):
        armor_class = {}
        for item in resp2['armor_class']:
            armor_class[item['type']] = item['value']
        monster_data['armor_class'] = armor_class

    if resp2.get('damage_vulnerabilities'):
        monster_data['damage_vulnerabilites'] = resp2['damage_vulnerabilities']

    if resp2.get('damage_resistances'):
        monster_data['damage_resistances'] = resp2['damage_resistances']
    
    if resp2.get('damage_immunities'):
        monster_data['damage_immunities'] = resp2['damage_immunities']

    if resp2.get('condition_immunities'):
        monster_data['condition_immunities'] = [item['name'] for item in resp2['condition_immunities']]

    if resp2.get('special_abilites'):
        special = {}
        for items in resp2['special_abilities']:
            special['name'] = items['name']
            special['desc'] = items['desc']
            if 'damage' in items:
                special['damage'] = items['damage']
            if 'dc' in items:
                dc = {}
                dc['name'] = items['dc']['dc_type']['name']
                dc['value'] = items['dc']['dc_value']
                special['dc'] = dc
        monster_data['special_abilities'] = special
    
    if resp2.get('actions'):
        for items in resp2['actions']:
            actions = {}
            actions['name'] = items['name']
            actions['desc'] = items['desc']
        monster_data['actions'] = actions

    if resp2.get('legendary_actions'):
        legendary = {}
        for items in resp2['legendary_actions']:
            legendary['name'] = items['name']
            legendary['action_desc'] = items['desc']
        monster_data['legendary_actions'] = legendary

    if resp2.get('forms'):
        monster_data['forms'] = [item['name'] for item in resp2['forms']]

    if resp2.get('reactions'):
        reactions = {}
        for item in resp2['reactions']:
            reactions['name'] = item['name']
            reactions['desc'] = item['desc']
        monster_data['reactions'] = reactions
    
    proficiency_monster = {}
    if resp2.get('proficiencies'):
        for items in resp2['proficiencies']:
            proficiency_monster[items['proficiency']['name']] = items['value']
        monster_data['proficiencies'] = proficiency_monster

    dict_of_monsters[entry['index']] = monster_data

pprint.pprint(dict_of_monsters)

{'aboleth': {'actions': {'desc': 'The aboleth targets one creature it can see '
                                 'within 30 ft. of it. The target must succeed '
                                 'on a DC 14 Wisdom saving throw or be '
                                 'magically charmed by the aboleth until the '
                                 'aboleth dies or until it is on a different '
                                 'plane of existence from the target. The '
                                 "charmed target is under the aboleth's "
                                 "control and can't take reactions, and the "
                                 'aboleth and the target can communicate '
                                 'telepathically with each other over any '
                                 'distance.\n'
                                 'Whenever the charmed target takes damage, '
                                 'the target can repeat the saving throw. On a '
                       

Here, the API endpoint `equipment-categories` is processed.

In [24]:
url = 'https://www.dnd5eapi.co/api/2014/equipment-categories'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_equipment_categories = {}
category_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    category_data = {
        'name' : name
    }

    if resp2.get('equipment'):
        category_data['type'] = [item['name'] for item in resp2['equipment']]

    dict_of_equipment_categories[entry['index']] = category_data

pprint.pprint(dict_of_equipment_categories)

{'adventuring-gear': {'name': 'Adventuring Gear',
                      'type': ['Abacus',
                               'Acid (vial)',
                               "Alchemist's fire (flask)",
                               'Arrow',
                               'Blowgun needle',
                               'Crossbow bolt',
                               'Sling bullet',
                               'Alms box',
                               'Amulet',
                               'Antitoxin (vial)',
                               'Backpack',
                               'Ball bearings (bag of 1,000)',
                               'Barrel',
                               'Basket',
                               'Bedroll',
                               'Bell',
                               'Blanket',
                               'Block and tackle',
                               'Block of incense',
                               'Book',
                               'B

Finally, the API endpoint `backgrounds` is processed. Due to copyright restrictions, only a single character background is available in the API. As a consequence, the retrieved information is later stored within a dictionary named `example_character_background`.

In [25]:
url = 'https://www.dnd5eapi.co/api/2014/backgrounds'
response = requests.request('GET', url, headers=headers, data=payload)
resp = response.json()

dict_of_backgrounds = {}
background_data = {}

for entry in resp['results']:
    name = entry['name']

    time.sleep(delay)
    
    response2 = requests.request('GET', url+f'/{entry['index']}', headers=headers, data=payload)
    resp2 = response2.json()

    background_data = {
        'name' : name
    }

    if resp2.get('starting_proficiencies'):
        background_data['starting_proficiencies'] = [item['name'] for item in resp2['starting_proficiencies']]

    if resp2.get('language_options'):
        background_data['language_options'] = resp2['language_options']['choose']

    if resp2.get('starting_equipment'):
        background_data['starting_equipment'] = [item['equipment']['name'] for item in resp2['starting_equipment']]

    if resp2.get('starting_equipment_options'):
        starting_equip_options = {}
        starting_equip_options['choose'] = [item['choose'] for item in resp2['starting_equipment_options']]
        starting_equip_options['equipment_options'] = [item['from']['equipment_category']['name'] for item in resp2['starting_equipment_options']]
        background_data['starting_equipment_options'] = starting_equip_options

    personality = { 'options':[] } 
    personality['amount_of_options'] = resp2['personality_traits']['choose'] 
    for item in resp2['personality_traits']['from']['options']: 
        personality['options'].append(item['string'])
    background_data['personality_traits'] = personality

    if resp2.get('feature'):
        feat_dict = {}
        feat_dict['name'] = resp2['feature']['name']
        feat_dict['desc'] =  ''.join(resp2['feature']['desc'])
        background_data['feature'] = feat_dict

    if resp2.get('ideals'):
        ideals = {}
        ideals['choose'] = resp2['ideals']['choose']
        ideal_option = {}
        ideals['possible_ideals'] = []
        for item in resp2['ideals']['from']['options']:
            ideal_option['desc'] = item['desc']
            ideal_option['alignments'] = [item['name'] for item in item['alignments']]
            ideals['possible_ideals'].append(ideal_option)
        background_data['ideals'] = ideals
    
    if resp2.get('bonds'):
        bonds = {}
        bonds['choose'] = resp2['bonds']['choose']
        bonds['bond_options'] = [item['string'] for item in resp2['bonds']['from']['options']]
        background_data['bonds'] = bonds

    if resp2.get('flaws'):
        flaws = {}
        flaws['choose'] = resp2['flaws']['choose']
        flaws['flaw_options'] = [item['string'] for item in resp2['flaws']['from']['options']]
        background_data['flaws'] = flaws

    dict_of_backgrounds[entry['index']] = background_data

pprint.pprint(dict_of_backgrounds)

{'acolyte': {'bonds': {'bond_options': ['I would die to recover an ancient '
                                        'relic of my faith that was lost long '
                                        'ago.',
                                        'I will someday get revenge on the '
                                        'corrupt temple hierarchy who branded '
                                        'me a heretic.',
                                        'I owe my life to the priest who took '
                                        'me in when my parents died.',
                                        'Everything I do is for the common '
                                        'people.',
                                        'I will do anything to protect the '
                                        'temple where I served.',
                                        'I seek to preserve a sacred text that '
                                        'my enemies consider heretical and '
 

Once all API endpoints have been processed into dictionaries, the data can be saved to the `api_data.json` file. To structure the JSON content, a new dictionary is created in which each previously generated dictionary is stored under a key corresponding to its respective endpoint. The JSON file is then opened in write mode, the constructed dictionary is written to it, and the file is automatically closed upon completion. For improved readability and to preserve a clear hierarchical structure, the JSON is formatted with an indentation of four spaces. Additionally, non-ASCII characters (e.g., accented letters or apostrophes) are correctly preserved without being escaped.

In [26]:
json_dict = {
    'rules': dict_of_rules,
    'rule_sections': dict_of_rule_sections,
    'races': dict_of_races,
    'subraces': dict_of_subraces,
    'classes': dict_of_classes,
    'subclasses': dict_of_subclasses,
    'skills': dict_of_skills,
    'feats': dict_of_feats,
    'languages': dict_of_languages,
    'ability_scores': dict_of_ability_scores,
    'traits': dict_of_traits,
    'proficiencies': dict_of_proficiencies,
    'features': dict_of_features,
    'example_character_background': dict_of_backgrounds,
    'conditions': dict_of_conditions,
    'equipment': dict_of_equipment,
    'equipment_categories': dict_of_equipment_categories,
    'weapon_properties': dict_of_weapon_properties,
    'magic_items': dict_of_magic_items,
    'magic_schools': dict_of_magic_schools,
    'damage_types': dict_of_damage_types,
    'spells': dict_of_spells,
    'monsters': dict_of_monsters
}

with open(file_path, 'w') as f:
    json.dump(
        json_dict,
        indent=4,
        ensure_ascii=False,
        fp=f
    )